# A random assortment of statistics and probability examples
__1. Birthday paradox__
<br>__2. Coupon collector's problem__
<br>__3. Poisson and exponential events probabilities__
<br>__4. Goodness of fit: chi-squared and k-s tests__
<br>__5. Jeep problem__

In [77]:
import numpy as np
import scipy.stats

### 1. Birthday paradox
Probability of having the same birthday.
<br>Some cool algebra tricks wth help of Taylor expansion of exp(x)
<br>https://betterexplained.com/articles/understanding-the-birthday-paradox/

In [15]:
### Probability given number people in room
days_in_year: int = 365
people_in_room: int = 6

prob_different = np.exp(-1*(people_in_room ** 2 / (2 * days_in_year)))
prob_same = 1 - prob_different
prob_same

0.04811882531956668

In [14]:
### How many people to get certain probability
desired_prob_same = 0.9

prob_different = 1 - desired_prob_same
# 0 = people_in_room^2 - people_in_room + 2 * days_in_year * np.ln(prob_different)
people_in_room = 1 + np.sqrt(1**2 - 4 * 1 * 2 * days_in_year * np.log(prob_different)) / (2 * 1)
people_in_room

42.00167213523923

### 2. Coupon collector's problem
Approach: work back from probabilities of getting the next unpossessed coupon.
<br>(Number of coupons required is reciprocal to probability of getting needed coupons.)
<br>Thanks to algebra, the summation simplifies to a quantity including the harmonic number of coupons.
<br>https://brilliant.org/wiki/coupon-collector-problem/

In [120]:
number_coupons: int = 10

def harmonic_num(x: int):
    harmonic_number: float = 0.0
    for i in range(x):
        harmonic_number += 1/(i + 1)
    return harmonic_number
number_times: float = number_coupons * harmonic_num(number_coupons)
np.round(number_times, 3)

29.29

### 3. Poisson events and exponential time between events
<br>References: Wiki articles for poisson and exponential distributions
<br>https://stats.stackexchange.com/questions/2092/relationship-between-poisson-and-exponential-distribution

__Baseball no-hitters events__
<br>[sandcat.middlebury.edu/econ/repec/mdl/ancoec/0717.pdf](sandcat.middlebury.edu/econ/repec/mdl/ancoec/0717.pdf)
<br>49 no-hitters in 40 years 1920-1959; over 280 months of baseball season
<br>lambda = 1.225 no-hitters per year
<br>lambda = 0.175 no-hitters per month

In [75]:
avg_event_rate = 49/280 # lambda: the number occurences per time period

### Poisson: probability of a given number of events occuring
number_events_per_time_interval_to_investigate: int = 0
    
def poisson_pdf(rate, number_events_to_check):
    return (rate ** number_events_to_check * 
            np.exp(-1 * rate) /
            np.math.factorial(number_events_to_check))
    
prob_of_that_number_of_events = poisson_pdf(rate = avg_event_rate, 
                                            number_events_to_check = number_events_per_time_interval_to_investigate)

prob_of_that_number_or_fewer_events = 0
for i in range(number_events_per_time_interval_to_investigate + 1):
    prob_of_that_number_or_fewer_events += \
    poisson_pdf(rate = avg_event_rate, 
                number_events_to_check = i)

print("Avg number events per time period:", 
      np.round(avg_event_rate, 3))
print("Number of events in that time period to investigate:", number_events_per_time_interval_to_investigate)
print("Prob_of_that_number_of_events:", 
      np.round(prob_of_that_number_of_events, 3))
print("Prob_of_that_number_or_fewer_events (including 0):", 
      np.round(prob_of_that_number_or_fewer_events, 3))

Avg number events per time period: 0.175
Number of events in that time period to investigate: 0
Prob_of_that_number_of_events: 0.839
Prob_of_that_number_or_fewer_events (including 0): 0.839


In [76]:
avg_event_rate = 49/280 # lambda: the number occurences per time period

### Exponential: time between events
time_to_investigate = 1/7 # 7 months in mlb season

prob_of_that_exact_time_between_occurences = avg_event_rate * np.exp(-1 * avg_event_rate * time_to_investigate)
prob_of_time_between_occurences = 1 - np.exp(-1 * avg_event_rate * time_to_investigate) # this amount of time or less

avg_time_between_events = 1 / avg_event_rate
median_time_between_events = np.log(2) / avg_event_rate
median_time_between_events

print("Avg number events per time period:", 
      np.round(avg_event_rate, 3))
print("Time between events to investigate:", 
      np.round(time_to_investigate, 3))
# print("Prob_density_at_that_exact_time_between_occurences:", 
#       np.round(prob_of_that_exact_time_between_occurences, 3))
print("Prob_of_that_time_or_less_between_events (including 0):", 
      np.round(prob_of_time_between_occurences, 3))
print("Avg_time_between_events:", 
      np.round(avg_time_between_events, 3))
print("Probability of avg time or less between events:", 
      np.round(1 - np.exp(-1 * avg_event_rate * avg_time_between_events), 3))
print("Median_time_between_events", 
      np.round(median_time_between_events, 3))

Avg number events per time period: 0.175
Time between events to investigate: 0.143
Prob_of_that_time_or_less_between_events (including 0): 0.025
Avg_time_between_events: 5.714
Probability of avg time or less between events: 0.632
Median_time_between_events 3.961


### 4. Goodness of fit
Chi-squared test for deviation from expected distribution
<br>Only for binned/categorical data
<br>[sandcat.middlebury.edu/econ/repec/mdl/ancoec/0717.pdf](sandcat.middlebury.edu/econ/repec/mdl/ancoec/0717.pdf)
<br>https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html

In [84]:
observed_array = [11, 15, 9, 4, 1]
expected_array = [11.75, 14.39, 8.8164, 3.6, 1.1025]
dof_adjustment = 1 # +1 if also estimating mean for poisson from sample data

print("If pvalue below 0.05, observations do not belong to expected distribution.")
print("P-value =",
      scipy.stats.chisquare(f_obs = observed_array,
                            f_exp = expected_array,
                            ddof = dof_adjustment)[1])

If pvalue below 0.05, observations do not belong to expected distribution.
P-value = 0.9878024170265235


Goodness of fit for continuous variables
<br>https://stats.stackexchange.com/questions/76350/goodness-of-fit-for-continuous-variables

Scipy kstest
<br>https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kstest.html

List of continuous distributions in scipy
<br>https://docs.scipy.org/doc/scipy/reference/stats.html#module-scipy.stats

In [94]:
scipy.stats.kstest(rvs = scipy.stats.uniform.rvs(size=1000, random_state = 12345),
                  cdf = "norm",
                  #args = (1, 1),
                   # N = 20,
                  alternative = 'two-sided',
                  mode = 'approx')

KstestResult(statistic=0.5000427891604897, pvalue=9.655302994802462e-232)

Named tuple
<br>https://docs.python.org/2/library/collections.html

In [101]:
import collections
norm_args = collections.namedtuple('norm_args', ['loc', 'scale'])

__main__.norm_args

In [102]:
scipy.stats.kstest(rvs = scipy.stats.norm.rvs(loc = 1,
                                              scale = 1,
                                              size = 1000, 
                                              random_state = 12345),
                  cdf = "norm",
                  args = norm_args(loc=1, scale=1),
                   # N = 20,
                  alternative = 'two-sided',
                  mode = 'approx')

KstestResult(statistic=0.02526609656624318, pvalue=0.5457961444206735)

In [103]:
uniform_args = collections.namedtuple('uniform_args', ['loc', 'scale'])

In [107]:
scipy.stats.kstest(rvs = scipy.stats.uniform.rvs(loc = 1,
                                                 scale = 1,
                                                 size=1000, 
                                                 random_state = 12345),
                  cdf = "uniform",
                  args = uniform_args(loc=1, scale=1),
                   # N = 20,
                  alternative = 'two-sided',
                  mode = 'approx')

KstestResult(statistic=0.014856419814211264, pvalue=0.9800639362901664)

__5. Jeep problem__
<br>https://en.wikipedia.org/wiki/Jeep_problem

In [132]:
desert_width = 5

def dist_explore(num_trips):
    return harmonic_num(x = num_trips) / 2

def dist_cross(num_trips):
    final_trip_distance = 0.0
    for i in range(num_trips):
        final_trip_distance += 1 / (2 * (i + 1) - 1)
    return final_trip_distance

resultant_distance = 0.0
n_trips_to_explore = 0
while resultant_distance < desert_width:
    n_trips_to_explore += 1
    resultant_distance = dist_explore(num_trips = n_trips_to_explore)

resultant_distance = 0.0
n_trips_to_cross = 0
while resultant_distance < desert_width:
    n_trips_to_cross += 1
    resultant_distance = dist_cross(num_trips = n_trips_to_cross)

print(n_trips_to_explore, "trips needed to explore")
print(n_trips_to_cross, "trips needed to cross")

12367 trips needed to explore
3092 trips needed to cross


In [131]:
dist_cross(3)

1.5333333333333332